In [1]:
# Crawling에 필요한 module 선언
import time

from selenium import webdriver

# (201105) 추가한 모듈, BeautifulSoup4 사용 추가
from bs4 import BeautifulSoup

# lxml
from lxml import html

# pandas
import pandas as pd

# requests, json, re
import requests
import json

In [2]:
# 유튜브 정식 카테고리 k, v dictionary
youtube_category_dict = {
    'Film & Animation' : 1,
    'Autos & Vehicles' : 2,
    'Music' : 10,
    'Pets & Animals' : 15,
    'Sports' : 17,
    'Short Movies' : 18,
    'Travel & Events' : 19,
    'Gaming' : 20,
    'Videoblogging' : 21,
    'People & Blogs' : 22,
    'Comedy' : 23,
    'Entertainment' : 24,
    'News & Politics' : 25,
    'Howto & Style' : 26,
    'Education' : 27,
    'Science & Technology' : 28,
    'Movies' : 30,
    'Anime/Animation' : 31,
    'Action/Adventure' : 32,
    'Classics' : 33,
    'Documentary' : 35,
    'Drama' : 36,
    'Family' : 37,
    'Foreign' : 38,
    'Horror' : 39,
    'Sci-Fi/Fantasy' : 40,
    'Thriller' : 41,
    'Shorts' : 42,
    'Shows' : 43,
    'Trailers' :44
}

# 샘플 키워드 리스트
animal_list = ['동물', '반려동물', '고양이', '강아지']
music_list = ['음악','노래', 'OST']
game_list = ['게임','롤','배그']
politics_list = ['뉴스','아침뉴스']
vlog_list = ['브이로그','vlog']
travel_list = ['여행','해외여행', '국내여행']
sports_list = ['스포츠','SPOTV', '축구']
comedy_list = ['개그','코미디']
entertainment_list = ['예능']
movie_list = ['영화','영화리뷰', '영화추천']
education_list = ['토익', '수능', '반려동물 교육']
science_list = ['컴퓨터', '조립', '우주', '가상화','실험']

# 모든 키워드를 다 합친 리스트
all_search_keyword_list = sports_list + movie_list + vlog_list + music_list + comedy_list + game_list + politics_list + travel_list + animal_list + entertainment_list + education_list + science_list
#all_search_keyword_list = science_list
print("All keywords list: ", all_search_keyword_list)

# video_id_list
# sports_video_id_list = []
# movie_video_id_list = []
# kids_video_id_list = []
# music_video_id_list = []
# comedy_video_id_list = []
# game_video_id_list = []
# politics_video_id_list = []
# cook_video_id_list = []
# animal_video_id_list = []

# 샘플 키워드 딕셔너리
# search_dict = {
#     'Sports' : [sports_list, sports_video_id_list]
#     'Movie' : [movie_list, movie_video_id_list]
#     'Kids' : [kids_list, kids_video_id_list]
#     'Music' : [music_list, music_video_id_list]
#     'Comedy' : [comedy_list, comedy_video_id_list]
#     'Game' : [game_list, game_video_id_list]
#     'Politics' : [politics_list, politics_video_id_list]
#     'Cook' : [cook_list, cook_video_id_list]
#     'Animal' : [animal_list, animal_video_id_list]
# }

All keywords list:  ['스포츠', 'SPOTV', '축구', '영화', '영화리뷰', '영화추천', '브이로그', 'vlog', '음악', '노래', 'OST', '개그', '코미디', '게임', '롤', '배그', '뉴스', '아침뉴스', '여행', '해외여행', '국내여행', '동물', '반려동물', '고양이', '강아지', '예능', '토익', '수능', '반려동물 교육', '컴퓨터', '조립', '우주', '가상화', '실험']


In [3]:
# 샘플 키워드 리스트 #2

In [4]:
# 검색어 결과별로 비디오 id를 모아주는 함수 정의
# 리턴은 dictionary
def get_search_keyword_scrap_video_id(search_keyword_list):
    # 최종적으로 리턴할 dictionary 정의
    youtube_id_dict = {}

    for word in search_keyword_list:
        # youtube_id_list
        youtube_id_list = []
        
        # webdriver define
        driver_path = './chromedriver.exe'
        driver = webdriver.Chrome(driver_path)
        driver.implicitly_wait(30) # or bigger second
        
        # 열고자 하는 search url 정의
        url = "https://www.youtube.com/results?search_query=" + word+"&sp=EgIIAg%253D%253D"
        print("full url: ", url)

        # 해당 url로 창 열기
        driver.get(url)
        time.sleep(5)

        # 첫 동영상 시작 시 시작번호(index = 1)
        index = 1
        SCROLL_PAUSE_TIME = 2

        # infinite scroll
        while True:
            try:
                youtube_video_id_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer['+str(index)+']/div[1]/div/div[1]/div/h3/a'

                # 해당 페이지의 8개의 영상을 모두 확인했다면 그다음으로 가기위해 스크롤
                if index % 4 == 0:
                    # 2초 대기 (대기 시간은 크롤러의 정밀도에 따라 조정 가능)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    last_height = driver.execute_script("return document.body.scrollHeight")

                    while True:
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(SCROLL_PAUSE_TIME)

                        new_height = driver.execute_script("return document.body.scrollHeight")

                        if new_height == last_height:
                            break

                        last_height = new_height


                youtube_title_link = driver.find_element_by_xpath(youtube_video_id_xpath)
                youtube_id = youtube_title_link.get_attribute('href')
                youtube_id = youtube_id.split("v=")[1].split("&")[0]
                youtube_id_list.append(youtube_id)

                # 리스트에 들어간 데이터 중간 확인
                print("index_no: ", index)
                print("youtube_id: ", youtube_id)

                index += 1

            except Exception as e:
                print()
                print("Exception: ", e)
                break
        # dictionary에 검색 키워드 별 리스트 추가
        youtube_id_dict[word] = youtube_id_list
        # youtube_id 수집이 끝난 브라우저 닫기        
        driver.close()
    
    return youtube_id_dict

In [5]:
youtube_id_dict = get_search_keyword_scrap_video_id(all_search_keyword_list)
print("youtube_id_dict: ", youtube_id_dict)

full url:  https://www.youtube.com/results?search_query=스포츠&sp=EgIIAg%253D%253D
index_no:  1
youtube_id:  S_dBHSLMhA4
index_no:  2
youtube_id:  vhjtVkV29ac
index_no:  3
youtube_id:  6Ir2aM8XB1Y
index_no:  4
youtube_id:  sBufHUC_oNU
index_no:  5
youtube_id:  hKYcClA0HEc
index_no:  6
youtube_id:  K9mvoRdEmTM
index_no:  7
youtube_id:  0mCWdhLar4c
index_no:  8
youtube_id:  E01U52lx5kk
index_no:  9
youtube_id:  7rugL3f5oqM
index_no:  10
youtube_id:  nJHWDH8HzPs
index_no:  11
youtube_id:  YSZCDOZqm4U
index_no:  12
youtube_id:  Xxw5cCMB-14
index_no:  13
youtube_id:  nKEAkyeD-U0
index_no:  14
youtube_id:  DEbV--shi_o
index_no:  15
youtube_id:  b0zBgfqkei8
index_no:  16
youtube_id:  zTDtUjtnhbs
index_no:  17
youtube_id:  DsJiqrck_g0
index_no:  18
youtube_id:  LHtItyCdyVw
index_no:  19
youtube_id:  v0poAPCTQNA
index_no:  20
youtube_id:  iC7-gUwV9sE

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div

index_no:  16
youtube_id:  FUvVBOFk0DU
index_no:  17
youtube_id:  9xnF9l-yMVg
index_no:  18
youtube_id:  D8eTmsOr68M
index_no:  19
youtube_id:  TMKVofBdS2k
index_no:  20
youtube_id:  Y6xVyNzOqMQ

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[21]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87.0.4280.88)

full url:  https://www.youtube.com/results?search_query=음악&sp=EgIIAg%253D%253D
index_no:  1
youtube_id:  WrvLWVQqnRI
index_no:  2
youtube_id:  aXNLrYRAhrw
index_no:  3
youtube_id:  S51A-mRE5jc
index_no:  4
youtube_id:  B0AUmS6QXE8
index_no:  5
youtube_id:  9riX2CVKGDQ
index_no:  6
youtube_id:  pU2G4r5-nnY
index_no:  7
youtube_id:  iyShYU5IWZg
index_no:  8
youtube_id:  fQKHBqhHt38
index_no:  9
youtube_id:  Yl0FNKSFmUA
index_no:  10
youtube_id:  2

index_no:  12
youtube_id:  n2Z3Ajm1KJo
index_no:  13
youtube_id:  OpxkM4XCF0I
index_no:  14
youtube_id:  QS3M4jyEA_0
index_no:  15
youtube_id:  CPuQeH5nBYE
index_no:  16
youtube_id:  H-y5HiD7ZG8
index_no:  17
youtube_id:  IMlPpnIQ8-4
index_no:  18
youtube_id:  ApkTG5lfbPE
index_no:  19
youtube_id:  oHAGg5eIzNE
index_no:  20
youtube_id:  blshi5U-Q1c

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[21]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87.0.4280.88)

full url:  https://www.youtube.com/results?search_query=배그&sp=EgIIAg%253D%253D
index_no:  1
youtube_id:  XE_x4EhOK8w
index_no:  2
youtube_id:  6VjOBIi7YJE
index_no:  3
youtube_id:  FwiiZciUPOk
index_no:  4
youtube_id:  364JfMq2pLs
index_no:  5
youtube_id:  u9zvVfA0ixQ
index_no:  6
youtube_id:

index_no:  8
youtube_id:  nlxX4UYb8sA
index_no:  9
youtube_id:  DJuTpIaAERA
index_no:  10
youtube_id:  pFmfjTT4pCE
index_no:  11
youtube_id:  l9epHQQbnaI
index_no:  12
youtube_id:  YbEb_keFtWc
index_no:  13
youtube_id:  EO5MIioH9jk
index_no:  14
youtube_id:  N6XvCYujFcc
index_no:  15
youtube_id:  WFfeSpGvH4s
index_no:  16
youtube_id:  nteKuiZdmB8
index_no:  17
youtube_id:  sqMHL4pBmsk
index_no:  18
youtube_id:  Ni3Qa14h9qs
index_no:  19
youtube_id:  OmVSqkl-Shc
index_no:  20
youtube_id:  Kxb6XcTdYus

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[21]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87.0.4280.88)

full url:  https://www.youtube.com/results?search_query=반려동물&sp=EgIIAg%253D%253D
index_no:  1
youtube_id:  bDRVQ-AWJTs
index_no:  2
youtube

full url:  https://www.youtube.com/results?search_query=컴퓨터&sp=EgIIAg%253D%253D

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[1]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87.0.4280.88)

full url:  https://www.youtube.com/results?search_query=조립&sp=EgIIAg%253D%253D
index_no:  1
youtube_id:  MZT-gLVEjBk
index_no:  2
youtube_id:  YX2FflZbpgA
index_no:  3
youtube_id:  6mHgDd6N8Jc
index_no:  4
youtube_id:  pkuB-00e928
index_no:  5
youtube_id:  PCiKvnisXW8
index_no:  6
youtube_id:  -2qXqAKyZMg
index_no:  7
youtube_id:  8MiGGGjFp14
index_no:  8
youtube_id:  YxWMh8H-pXw
index_no:  9
youtube_id:  kjf74W6p1J8
index_no:  10
youtube_id:  L8HzR6Gm_jQ
index_no:  11
youtube_id:  OlrJcH_1-oE
index_no:  12
youtube_id:  2SUGB8u3qXk
index_no:  13
youtube_id:  

In [6]:
# 특정 검색어 키워드 기반으로 크롤링 해 주는 크롤러 정의
def scrap_search_detail_video_data(video_id_dict):
    # 모든 결과를 누적해서 보여주는 df 정의
    youtube_raw_data_columns = ['title', 'authorNick','urlPath', 'thumbUrl', 'likeCnt', 'dislikeCnt',
                                'followCnt', 'viewCnt', 'duration', 'utbCategory', 'utbCateSN', 'isFamilyFriendly',
                                'titleHashTags', 'descHashTags', 'utbVideoID', 'authorID', 'yUploadDateTime', 'keywords', 'searchKeyword']
    
    # 모든 DF를 누적할 DataFrame정의
    total_df = pd.DataFrame(columns = youtube_raw_data_columns)
    
    # 각 video_id에 해당하는 유튜브 영상 별로 상세 데이터 수집
    for keyword, youtube_id_list in video_id_dict.items():
        
        print("==========outer loop==========")
        print("keyword: ", keyword)
        print("video_id_list: ", youtube_id_list)
        
        # 수집할 데이터의 리스트
        title_list = []
        author_list = []
        #description_list = []
        url_list = []
        thumb_url_list = []
        like_cnt_list = []
        dislike_cnt_list = []
        follow_cnt_list = []
        view_cnt_list = []
        duration_list = []
        youtube_category_list = []
        youtube_category_sn_list = []
        is_family_friendly_list = []
        title_hash_tags_list = []
        description_hash_tags_list = []
        video_id_list = [] 
        author_id_list = []
        yupload_time_list = []
        keywords_list = []
        searchKeyword_list = []
        
        # youtube_raw_data를 받을 dictionary 정의
        youtube_raw_data = {}
        
        for video_id in youtube_id_list:
            print("==========inner loop==========")
            
            # webdriver 초기화
            driver_path = './chromedriver.exe'
            driver = webdriver.Chrome(driver_path)
            driver.implicitly_wait(100) # or bigger second

            # origin_url 정의
            origin_url = 'https://www.youtube.com/watch?v='

            # 넣어주는 url 값
            url = origin_url + video_id

            # scriptTag에서 긁어지는 데이터
            script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'

            # meta tag에서 긁어지는 데이터 -> soup.find, soup.findAll 이용해서 긁기
            request = requests.get(url)
            soup = BeautifulSoup(request.text, 'lxml')

            # ytInitialData에서 긁어지는 데이터 -> 복잡한 계층의 json 데이터 기반
            yt_initial_data_xpath = '/html/body/script[18]'
            
            driver.get(url)
            time.sleep(5)
            
            # 데이터를 담을 변수 정의(scriptTag, metadata, ytInitialData)
            try:
                # Data 크롤링
                # 각 유형별로 다른 로직으로 데이터를 크롤링한다
                # script tag id=scriptTag의 내부 HTML 내용을 가져온다(scriptTag data crawling)
                script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

                script_tag_json = json.loads(script_tag_str)

                time.sleep(5)

                # print("script_tag_json: ", script_tag_json)
                # print("script_tag_json_type: ", type(script_tag_json))

                # Data result test
                title = script_tag_json['name']
                author = script_tag_json['author']
                #description = script_tag_json['description']
                thumb_url = script_tag_json['thumbnailUrl'][0]
                view_cnt = int(script_tag_json['interactionCount'])
                duration = int(script_tag_json['duration'][2:len(script_tag_json['duration'])-1].replace(",", ""))
                youtube_category = script_tag_json['genre']
                # addition : youtube_category_sn
                youtube_category_sn = youtube_category_dict[youtube_category]

                print()
                print("=" * 20, "Youtube Raw Data Descriptions", "=" * 20)
                print("title: ", title)
                print("author: ", author)
                #print("description: ", description)
                print("thumb_url: ", thumb_url)
                print("view_cnt: ", view_cnt, " type: ", type(view_cnt))
                print("duration: ", duration, " type: ", type(duration))
                print("youtube_category: ", youtube_category)
                print("youtube_category_id: ", youtube_category_sn)

                # meta tag data crawling
                is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
                author_id = soup.find("meta", {"itemprop" : "channelId"})['content']
                yupload_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
                keywords = ",".join([tag['content'].replace("#", "") for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

                print("is_family_friendly: ", is_family_friendly)
                print("video_id: ", video_id)
                print("author_id: ", author_id)
                print("yupload_time: ", yupload_time)
                print("keywords: ", keywords)

                # ytInitialData crawling
                yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
                yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('var ytInitialData =', '').strip()
                # print("yt_initial_data: ", yt_initial_data_str)

                time.sleep(5)

                # data to json
                # json loading이 간헐적으로 안되는 현상 수정 필요
                yt_initial_data_json = json.loads(yt_initial_data_str)

                # print("yt_initial_data_json: ", yt_initial_data_json)

                # video_info_renderer
                video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
                video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
                video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

                # print("video_primary_info_renderer: ", video_primary_info_renderer)
                # print("video_secondary_info_renderer: ", video_secondary_info_renderer)

                # 좋아요 & 싫어요
                like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

                print("like_dislike_cnt: ", like_dislike_cnt)

                like_cnt = int(like_dislike_cnt.split(" / ")[0].replace(",", ""))
                dislike_cnt = int(like_dislike_cnt.split(" / ")[1].replace(",", ""))

                print("like_cnt: ", like_cnt, " type: ", type(like_cnt))
                print("dislike_cnt: ", dislike_cnt, " type: ", type(dislike_cnt))

                # 제목 타이틀 해시태그
                # 타이틀 해시태그가 없는 경우 빈문자열로 처리
                raw_title_hash_tags = video_primary_info_renderer.get('superTitleLink', "")
                if raw_title_hash_tags != "":
                    title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
                    title_hash_tag_list = []
                    for tag in title_hash_tags:
                        if tag['text'] not in [" ", ""]:
                            title_hash_tag_list.append(tag['text'][1:])
                            # print("title_hash_tag: ", tag['text'][1:])
                    title_hash_tags_str = ",".join(title_hash_tag_list)
                else:
                    title_hash_tags_str = ""
                print("title_hash_tags_str: ", title_hash_tags_str)

                # 내용 해시태그
                description_hash_tags = video_secondary_info_renderer['description']['runs']
                description_hash_tag_list = []
                for tag in description_hash_tags:
                    if tag['text'] not in [" ", ""]:
                        if "#" in tag['text']:
                            description_hash_tag_list.append(tag['text'][1:])
                            # print("description_hash_tag: ", tag['text'][1:])

                description_hash_tags_str = ",".join(description_hash_tag_list)
                print("description_hash_tags_str: ", description_hash_tags_str)

                # 구독자 수
                # 구독자 수가 없는 경우에 대한 예외처리(KeyError) -> 0 으로 처리
                raw_follow = video_secondary_info_renderer['owner']['videoOwnerRenderer'].get('subscriberCountText', "")
                if raw_follow != "":
                    raw_follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text'][4:]
                    follow_cnt = raw_follow_cnt[:len(raw_follow_cnt)-1]
                    print("follow_cnt: ", follow_cnt)

                    # 구독자 수 수치화 필터링
                    num_count_dict = {
                        '천' : 1000,
                        '만' : 10000,
                        '억' : 100000000,
                        '조' : 1000000000000
                    }

                    # print("follow_cnt: ", follow_cnt)
                    # print("follow_num_cnt: ", follow_cnt[-1])
                    # 천, 만, 억, 조 에 걸리는 경우 -> 그 만큼 곱해서 리턴, 그게 아닐 경우 그냥 정수화 후 리턴
                    if follow_cnt[-1] in num_count_dict.keys():
                        follow_cnt_num = int(float(follow_cnt[:len(follow_cnt)-1]) * num_count_dict[follow_cnt[-1]])
                    else:
                        follow_cnt_num = int(follow_cnt)
                else:
                    follow_cnt_num = 0

                print("follow_cnt_num: ", follow_cnt_num, " type: ", type(follow_cnt_num))

                # 예외처리에 걸리지 않고 무사히 넘어왔다면 list에 데이터가 담긴다
                # scriptTag data appending
                title_list.append(title)
                author_list.append(author)
                #description_list.append(description)
                url_list.append(url)
                thumb_url_list.append(thumb_url)
                view_cnt_list.append(view_cnt)
                duration_list.append(duration)
                youtube_category_list.append(youtube_category)
                youtube_category_sn_list.append(youtube_category_sn)

                # meta tag data appending
                is_family_friendly_list.append(is_family_friendly)
                video_id_list.append(video_id)
                author_id_list.append(author_id)
                yupload_time_list.append(yupload_time)
                keywords_list.append(keywords)
                searchKeyword_list.append(keyword)

                # yInitialData data appending
                like_cnt_list.append(like_cnt)
                dislike_cnt_list.append(dislike_cnt)
                title_hash_tags_list.append(title_hash_tags_str)
                description_hash_tags_list.append(description_hash_tags_str)
                follow_cnt_list.append(follow_cnt_num)
                
            except Exception as e:
                print()
                print("Exception: ", e)
                driver.close()
                continue
            
            print("=" * 20, "Youtube Raw Data Descriptions Ended", "=" * 20, "\n")
            
            driver.close()

        # 얻어온 데이터를 dictionary에 저장
        youtube_raw_data['title'] = title_list
        youtube_raw_data['authorNick'] = author_list
        #youtube_raw_data['description'] = description_list
        youtube_raw_data['urlPath'] = url_list
        youtube_raw_data['thumbUrl'] = thumb_url_list
        youtube_raw_data['likeCnt'] = like_cnt_list
        youtube_raw_data['dislikeCnt'] = dislike_cnt_list
        youtube_raw_data['followCnt'] = follow_cnt_list
        youtube_raw_data['viewCnt'] = view_cnt_list
        youtube_raw_data['duration'] = duration_list
        youtube_raw_data['utbCategory'] = youtube_category_list
        youtube_raw_data['utbCateSN'] = youtube_category_sn_list
        youtube_raw_data['isFamilyFriendly'] = is_family_friendly_list
        youtube_raw_data['titleHashTags'] = title_hash_tags_list
        youtube_raw_data['descHashTags'] = description_hash_tags_list
        youtube_raw_data['utbVideoID'] = video_id_list
        youtube_raw_data['authorID'] = author_id_list
        youtube_raw_data['yUploadDateTime'] = yupload_time_list
        youtube_raw_data['keywords'] = keywords_list
        youtube_raw_data['searchKeyword'] = searchKeyword_list
        
        print("youtube_raw_data: ", youtube_raw_data)
        
        # DataFrame 생성
        df = pd.DataFrame(youtube_raw_data)
        
        total_df.append(df, ignore_index = True)

        # 모아둔 DataFrame을 csv로 변환
        df.to_csv("./1221_youtube_.csv", sep="|", mode="a",header= False,index=False, encoding='utf-8-sig')

    return total_df

In [7]:
scrap_search_detail_video_data(youtube_id_dict)

==========outer loop==========
keyword:  스포츠
video_id_list:  ['S_dBHSLMhA4', 'vhjtVkV29ac', '6Ir2aM8XB1Y', 'sBufHUC_oNU', 'hKYcClA0HEc', 'K9mvoRdEmTM', '0mCWdhLar4c', 'E01U52lx5kk', '7rugL3f5oqM', 'nJHWDH8HzPs', 'YSZCDOZqm4U', 'Xxw5cCMB-14', 'nKEAkyeD-U0', 'DEbV--shi_o', 'b0zBgfqkei8', 'zTDtUjtnhbs', 'DsJiqrck_g0', 'LHtItyCdyVw', 'v0poAPCTQNA', 'iC7-gUwV9sE']
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  🛹스포츠 게임도 '갓겜'이 될 수 있다 | '토니 호크의 프로 스케이터'🛹
author:  Beyond Sports
thumb_url:  https://i.ytimg.com/vi/S_dBHSLMhA4/maxresdefault.jpg
view_cnt:  7283  type:  <class 'int'>
duration:  397  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  true
video_id:  S_dBHSLMhA4
author_id:  UCJronx9CcSDgRAnHrVW2BNA
yupload_time:  2020-12-19
keywords:  프로스케이터,pro skater,프로 스케이터,토니호크,액티비전,스케이트보드 게임,스케이트 게임

Exception:  Unterminated string starting at: line 1 column 131836 (char 131835)
==========in

like_dislike_cnt:  32 / 1
like_cnt:  32  type:  <class 'int'>
dislike_cnt:  1  type:  <class 'int'>
title_hash_tags_str:  프로토,베트맨,해외축구
description_hash_tags_str:  프로토,베트맨,해외축구,해축
follow_cnt:  7.54천
follow_cnt_num:  7540  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  스포츠토토 축구토토_축구분석 12월 21일 프리미어리그 EPL 세리에A
author:  토토 김박사
thumb_url:  https://i.ytimg.com/vi/Xxw5cCMB-14/hqdefault.jpg
view_cnt:  2229  type:  <class 'int'>
duration:  644  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  true
video_id:  Xxw5cCMB-14
author_id:  UC0tXq2xhHoVGzRwFBEE60wQ
yupload_time:  2020-12-19
keywords:  스포츠토토,토토,프로토,배트맨토토,스포츠분석,중국축구,축구토토,축구분석,중국슈퍼리그,유럽축구,유럽리그,승무패,언오버,토토잘하는법,스포츠토토잘하는법,유럽토토

Exception:  Unterminated string starting at: line 1 column 38778 (char 38777)
==========inner loop==========

==


==================== Youtube Raw Data Descriptions ====================
title:  [20/21 PL 14R] C.팰리스 vs 리버풀 H/L
author:  SPOTV
thumb_url:  https://i.ytimg.com/vi/BRaHqc7BV2A/maxresdefault.jpg
view_cnt:  459758  type:  <class 'int'>
duration:  711  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  true
video_id:  BRaHqc7BV2A
author_id:  UCtm_QoN2SIxwCE-59shX7Qg
yupload_time:  2020-12-19
keywords:  SPOTV,스포티비,프리미어리그,살라,마네,피르미누,C.팰리스,미나미노,파비뉴
like_dislike_cnt:  3,537 / 142
like_cnt:  3537  type:  <class 'int'>
dislike_cnt:  142  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  147만
follow_cnt_num:  1470000  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  [20/21 PL 14R] 에버튼 vs 아스날 H/L
author:  SPOTV
thumb_url:  https://i.ytimg.com/vi/-LpwgvRDVQ0/ma

like_dislike_cnt:  172 / 4
like_cnt:  172  type:  <class 'int'>
dislike_cnt:  4  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  147만
follow_cnt_num:  1470000  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  [20/21 라리가] 14R AT마드리드 vs 엘체 H/L
author:  SPOTV
thumb_url:  https://i.ytimg.com/vi/c89OeDDKmzA/maxresdefault.jpg
view_cnt:  38425  type:  <class 'int'>
duration:  820  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  true
video_id:  c89OeDDKmzA
author_id:  UCtm_QoN2SIxwCE-59shX7Qg
yupload_time:  2020-12-19
keywords:  SPOTV,스포티비,AT마드리드,엘체,시메오네,수아레스,디에고코스타,바디아,보예
like_dislike_cnt:  436 / 9
like_cnt:  436  type:  <class 'int'>
dislike_cnt:  9  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  147만
follow_cn


==================== Youtube Raw Data Descriptions ====================
title:  [20/21 라리가] 14R 바르셀로나 vs 발렌시아 H/L
author:  SPOTV
thumb_url:  https://i.ytimg.com/vi/W4Vf7grYkuM/maxresdefault.jpg
view_cnt:  133633  type:  <class 'int'>
duration:  766  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  true
video_id:  W4Vf7grYkuM
author_id:  UCtm_QoN2SIxwCE-59shX7Qg
yupload_time:  2020-12-19
keywords:  SPOTV,스포티비,바르셀로나,리오넬 메시,그리즈만,이강인,가야

Exception:  Unterminated string starting at: line 1 column 67656 (char 67655)
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  그라운드의 악동 축구선수 TOP 10
author:  불양TV
thumb_url:  https://i.ytimg.com/vi/tNuuI6jAk_o/hqdefault.jpg
view_cnt:  21820  type:  <class 'int'>
duration:  390  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  true
video_id:  tNuuI6jAk_o
author_id:  UCoKXVjv23HMyzA9cZ_XB9Jw
yupload_time:  202

...


==================== Youtube Raw Data Descriptions ====================
title:  현실을 가상화하는 시대 메타버스
author:  MAXST
thumb_url:  https://i.ytimg.com/vi/jm1vcFgla48/maxresdefault.jpg
view_cnt:  41  type:  <class 'int'>
duration:  73  type:  <class 'int'>
youtube_category:  Science & Technology
youtube_category_id:  28
is_family_friendly:  true
video_id:  jm1vcFgla48
author_id:  UCFf4WApLJVObJpJp7-T9qGg
yupload_time:  2020-12-20
keywords:  맥스트,MAXST,AR,증강현실,SLAM,ARSDK

Exception:  Unterminated string starting at: line 1 column 60835 (char 60834)
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  [온라인 세미나] MICROSOFT 가상화 라이선스 (서버, 데스크톱, 클라우드 등)
author:  스위퍼 SWeeper
thumb_url:  https://i.ytimg.com/vi/RikTKSajE9w/maxresdefault.jpg
view_cnt:  4  type:  <class 'int'>
duration:  2827  type:  <class 'int'>
youtube_category:  People & Blogs
youtube_category_id:  22
is_family_friendly:  true
video_id:  RikTKSajE9w
author_id:  UCsFcXAICnXm7Q


Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script"}
  (Session info: chrome=87.0.4280.88)

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  비트코인 초보자도 쉽게 적용할 수 있는 툴 공개합니다.  #비트코인 #암호화폐 #가상화폐 #알트코인 #이더리움 #반감기
author:  에임리치TV - 비트코인,주식
thumb_url:  https://i.ytimg.com/vi/d_Bv9OYseJs/maxresdefault.jpg
view_cnt:  982  type:  <class 'int'>
duration:  362  type:  <class 'int'>
youtube_category:  Education
youtube_category_id:  27
is_family_friendly:  true
video_id:  d_Bv9OYseJs
author_id:  UCt0MyoXAiA3nxpW29_t_IkA
yupload_time:  2020-12-19
keywords:  비트코인,반감기,비트코인전망,비트코인대박,비트코인레전드,비트코인하는법,비트코인채굴,비트코인반감기,가상화폐,가상화폐유튜버,가상화폐전망,가상화폐차트보는법,가상화폐단타,가상화폐채굴,암호화폐,암호화폐전망,암호화폐채굴,돈버는법,돈버는법학생,돈버는법부자되는법,돈버는법강의,재테크,재테크강의,라이트코인,알트코인,비트코인시세,암호화폐성공,암호화폐자료,블록체인,암호화폐하락,변동성,분석리포트,수익,투자자,에임리치,강세장,매수,매도,투자,부자되는법,


==================== Youtube Raw Data Descriptions ====================
title:  모동숲)눈덩이의 모든것을 실험한다! 눈덩이를 도구로 때리면 부서질까? 눈덩이를 2층에 올리고 1층의 절벽을 부순다면?
author:  옹이의 모동숲 이야기
thumb_url:  https://i.ytimg.com/vi/yxePDq3-9XU/maxresdefault.jpg
view_cnt:  30965  type:  <class 'int'>
duration:  493  type:  <class 'int'>
youtube_category:  Gaming
youtube_category_id:  20
is_family_friendly:  true
video_id:  yxePDq3-9XU
author_id:  UCCz3mJvnA8kA_QXWjWdXoEQ
yupload_time:  2020-12-19
keywords:  모동숲,동물의숲,모여봐요,모여봐요동물의숲,동숲,모여봐요동숲,플레이,영상,옹이,마이,닌텐도,닌텐도스위치,ACNH,Animal Crossing,New Horizons,あつ森,どうぶつの森,あつまれ どうぶつの森,눈덩이,눈사람,실험,겨울,눈,결정,얼음레시피
like_dislike_cnt:  800 / 20
like_cnt:  800  type:  <class 'int'>
dislike_cnt:  20  type:  <class 'int'>
title_hash_tags_str:  모여봐요동물의숲,겨울,눈사람
description_hash_tags_str:  모여봐요동물의숲,겨울,눈사람
follow_cnt:  5.62만
follow_cnt_num:  56200  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

================


Exception:  Unterminated string starting at: line 1 column 39044 (char 39043)
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  [그림자복도(실험)] 경종을 바구니에 집착시켜보자
author:  무아
thumb_url:  https://i.ytimg.com/vi/j5Z4V4rBWBA/maxresdefault.jpg
view_cnt:  5  type:  <class 'int'>
duration:  1804  type:  <class 'int'>
youtube_category:  Gaming
youtube_category_id:  20
is_family_friendly:  true
video_id:  j5Z4V4rBWBA
author_id:  UCc_ro_0VfjYFmdoUum7bPNA
yupload_time:  2020-12-20
keywords:  

Exception:  Unterminated string starting at: line 1 column 73304 (char 73303)
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  돌아가는 실험 유튜브를 믿지마세요!
author:  코코소다
thumb_url:  https://i.ytimg.com/vi/vK54pgHzoEY/maxresdefault.jpg
view_cnt:  4  type:  <class 'int'>
duration:  244  type:  <class 'int'>
youtube_category:  Pets & Animals
youtube_category_id:  15
is_family_friendly:  true
video_id:

,title,authorNick,urlPath,thumbUrl,likeCnt,dislikeCnt,followCnt,viewCnt,duration,utbCategory,utbCateSN,isFamilyFriendly,titleHashTags,descHashTags,utbVideoID,authorID,yUploadDateTime,keywords,searchKeyword
